In [1]:
from matplotlib import pyplot as plt
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from time import time
import mxnet as mx
import d2lzh as d2l
import random
import time
import sys 
import os

# 05. 深度学习计算

## 5.5 卷积神经⽹络(LeNet)
在`多层感知机的从零开始实现`一节里我们构造了一个含单隐藏层的多层感知机模型来对Fashion-MNIST数据集进行分类。每张图像高和宽均是28像素。我们将图像中的像素逐行展开，得到长度为784的向量，并输入进全连接层中。然而，这种分类方法有一定的局限性：
1. 图像在同一列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别
2. 对于大尺寸的输入图像，使用全连接层容易导致模型过大。假设输入是高和宽均为$1,000$像素的彩色照片(含3个通道)。即使全连接层输出个数仍是256，该层权重参数的形状也是$3,000,000\times 256$：它占用了大约3GB的内存或显存。这会带来过于复杂的模型和过高的存储开销

卷积层尝试解决这两个问题：
+ 卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别
+ 卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大

本节里将介绍一个早期用来识别手写数字图像的卷积神经网络：`LeNet`。这个名字来源于LeNet论文的第一作者`Yann LeCun`。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。

### 5.5.1 LeNet模型
`LeNet`分为卷积层块和全连接层块两个部分。

卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用`sigmoid`激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(`批量大小, 通道, 高, 宽`)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平(flatten)。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Sequential`类来实现`LeNet`模型。

In [2]:
net = nn.Sequential()
net.add(
    nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    # Dense会默认将(批量⼤⼩, 通道, ⾼, 宽)形状的输⼊转换成 
    # (批量⼤⼩, 通道 * ⾼ * 宽)形状的输⼊ 
    nn.Dense(120, activation='sigmoid'), 
    nn.Dense(84, activation='sigmoid'), 
    nn.Dense(10))

接下来我们构造⼀个⾼和宽均为28的单通道数据样本，并逐层进⾏前向计算来查看每个层的输出形状。

In [3]:
X = nd.random.uniform(shape=(1, 1, 28, 28))

net.initialize() 
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 6, 24, 24)
pool0 output shape:	 (1, 6, 12, 12)
conv1 output shape:	 (1, 16, 8, 8)
pool1 output shape:	 (1, 16, 4, 4)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


可以看到，在卷积层块中输⼊的⾼和宽在逐层减小。卷积层由于使⽤⾼和宽均为5的卷积核，从而将⾼和宽分别减小4，而池化层则将⾼和宽减半，但通道数则从1增加到16。全连接层则逐层减少输出个数，直到变成图像的类别数10。

LeNet架构图：
<img src="images/lenet.png" style="width:700px;"/>

### 5.5.2 获取数据和训练模型
使⽤Fashion-MNIST作为训练数据集实验LeNet模型：

In [4]:
batch_size = 256 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

因为卷积神经⽹络计算⽐多层感知机要复杂，建议使⽤GPU来加速计算。

```python
def try_gpu():
    # 本函数已保存在d2lzh包中⽅便以后使⽤
    try:
        ctx = mx.gpu() 
        _ = nd.zeros((1,), ctx=ctx) 
    except mx.base.MXNetError:
        ctx = mx.cpu() 
        return ctx

ctx = try_gpu() 
ctx
```

> jupyter运行在阿里云上，没有GPU，直接指定在CPU运行

In [5]:
ctx = mx.cpu()

相应地，我们对`softmax回归的从零开始实现`⼀节中描述的`evaluate_accuracy`函数略作修改。由于数据刚开始存在CPU使⽤的内存上，当`ctx`变量代表GPU及相应的显存时，我们通过`GPU计算`⼀节中介绍的`as_in_context`函数将数据复制到显存上，例如`gpu(0)`。

In [6]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
# 该函数将被逐步改进：它的完整实现将在`图像增⼴`⼀节中描述
def evaluate_accuracy(data_iter, net, ctx):
    acc_sum, n = nd.array([0], ctx=ctx), 0 
    for X, y in data_iter:
        # 如果ctx代表GPU及相应的显存，将数据复制到显存上
        X, y = X.as_in_context(ctx), y.as_in_context(ctx).astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum() 
        n += y.size 
    return acc_sum.asscalar() / n

我们同样对`softmax回归的从零开始实现`⼀节中定义的`train_ch3`函数略作修改，确保计算使⽤的数据和模型同在内存或显存上。

In [7]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx) 
            with autograd.record():
                y_hat = net(X) 
                l = loss(y_hat, y).sum() 
            l.backward() 
            trainer.step(batch_size) 
            y = y.astype('float32') 
            train_l_sum += l.asscalar() 
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, ' 'time %.1f sec' 
              % (epoch+1, train_l_sum/n, train_acc_sum/n, test_acc, time.time()-start))

我们重新将模型参数初始化到设备变量ctx之上，并使⽤Xavier随机初始化。损失函数和训练算法则依然使⽤交叉熵损失函数和小批量随机梯度下降。

In [8]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on cpu(0)
epoch 1, loss 2.3223, train acc 0.100, test acc 0.100, time 22.4 sec
epoch 2, loss 2.1511, train acc 0.175, test acc 0.546, time 20.3 sec
epoch 3, loss 1.0472, train acc 0.582, test acc 0.660, time 20.4 sec
epoch 4, loss 0.7924, train acc 0.691, test acc 0.717, time 20.4 sec
epoch 5, loss 0.6871, train acc 0.728, test acc 0.730, time 20.5 sec


尝试基于LeNet构造更复杂的⽹络来提⾼分类准确率。例如，调整卷积窗口⼤小、输出通道数、激活函数和全连接层输出个数。在优化⽅⾯，可以尝试使⽤不同的学习率、初始化⽅法以及增加迭代周期。

> 将激活函数换成`relu`，准确率提高到0.87左右


## 5.6 深度卷积神经⽹络(AlexNet)
在LeNet提出后的将近20年⾥，神经⽹络⼀度被其他机器学习⽅法超越，如SVM。 虽然LeNet可以在早期的小数据集上取得好的成绩，但是在更⼤的真实数据集上的表现并不尽如⼈意：
+ ⼀⽅⾯，神经⽹络计算复杂。早期GPU还没有⼤量普及，很难训练一个多层的、有⼤量参数的卷积神经⽹络
+ 另⼀⽅⾯，当年研究者还没有⼤量深⼊研究参数初始化和⾮凸优化算法等诸多领域，导致复杂的神经⽹络的训练通常较困难

我们在上⼀节看到，神经⽹络可以直接基于图像的原始像素进⾏分类。这种称为端到端的⽅法节省了很多中间步骤。然而，在很⻓⼀段时间⾥更流⾏的是研究者通过勤劳与智慧所设计并⽣成的⼿⼯特征。这类图像分类研究的主要流程是：
1. 获取图像数据集
2. 使⽤已有的特征提取函数⽣成图像的特征
3. 使⽤机器学习模型对图像的特征分类

当时认为的机器学习部分仅限最后这⼀步。如果那时候跟机器学习研究者交谈，他们会认为机器学习既重要⼜优美。优雅的定理证明了许多分类器的性质。机器学习领域⽣机勃勃、严谨而且极其有⽤。然而，如果跟计算机视觉研究者交谈，则是另外⼀幅景象。他们会告诉你图像识别⾥“不可告⼈”的现实是：计算机视觉流程中真正重要的是数据和特征。也就是说，使⽤较⼲净的数据集和较有效的特征甚⾄⽐机器学习模型的选择对图像分类结果的影响更⼤。

### 5.6.1 学习特征表⽰
既然特征如此重要，它该如何表⽰呢？

我们已经提到，在相当⻓的时间⾥，特征都是基于各式各样⼿⼯设计的函数从数据中提取的。事实上，不少研究者通过提出新的特征提取函数不断改进图像分类结果。这⼀度为计算机视觉的发展做出了重要贡献。

然而，另⼀些研究者则持异议。他们认为特征本⾝也应该由学习得来。他们还相信，为了表征⾜够复杂的输⼊，特征本⾝应该分级表⽰。持这⼀想法的研究者相信，多层神经⽹络可能可以学得数据的多级表征，并逐级表⽰越来越抽象的概念或模式。以图像分类为例，在多层神经⽹络中，图像的第⼀级的表⽰可以是在特定的位置和⻆度是否出现边缘；而第⼆级的表⽰说不定能够将这些边缘组合出有趣的模式，如花纹；在第三级的表⽰中，也许上⼀级的花纹能进⼀步汇合成对应物体特定部位的模式。这样逐级表⽰下去，最终，模型能够较容易根据最后⼀级的表⽰完成分类任务。需要强调的是，输⼊的逐级表⽰由多层模型中的参数决定，而这些参数都是学出来的。

尽管⼀直有⼀群执着的研究者不断钻研，试图学习视觉数据的逐级表征，然而很⻓⼀段时间⾥这些野⼼都未能实现。这其中有诸多因素值得我们⼀⼀分析。

##### 缺失要素⼀：数据
包含许多特征的深度模型需要⼤量的有标签的数据才能表现得⽐其他经典⽅法更好。早期⼤部分研究只基于小的公开数据集，其中许多数据集只有⼏百⾄⼏千张图像。这⼀状况在2010年前后兴起的⼤数据浪潮中得到改善。特别是，2009年诞⽣的ImageNet数据集包含了1,000⼤类物体，每类有多达数千张不同的图像。这⼀规模是当时其他公开数据集⽆法与之相提并论的。ImageNet数据集同时推动计算机视觉和机器学习研究进⼊新的阶段，使此前的传统⽅法不再有优势。

##### 缺失要素⼆：硬件
深度学习对计算资源要求很⾼。早期的硬件计算能⼒有限，这使训练较复杂的神经⽹络变得很困难。通⽤GPU的到来改变了这⼀格局。GPU都是为图像处理和计算机游戏设计的，尤其是针对⼤吞吐量的矩阵和向量乘法从而服务于基本的图形变换。这其中的数学表达与深度⽹络中的卷积层的表达类似。这使得GPU也在2010年前后开始被机器学习社区使⽤。

### 5.6.2 AlexNet
2012年，`AlexNet`横空出世。这个模型的名字来源于论⽂第⼀作者的姓名`Alex Krizhevsky`。`AlexNet`使⽤了8层卷积神经⽹络，并以很⼤的优势赢得了`ImageNet 2012图像识别挑战赛`。 它⾸次证明了学习到的特征可以超越⼿⼯设计的特征，从而⼀举打破计算机视觉研究的前状。

`AlexNet`与`LeNet`的设计理念⾮常相似，但也有显著的区别：
1. 与相对较小的`LeNet`相⽐，`AlexNet`包含8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。

    `AlexNet`第⼀层中的卷积窗口形状是$11 \times 11$。 因为ImageNet中绝⼤多数图像的⾼和宽均⽐MNIST图像的⾼和宽⼤10倍以上，ImageNet图像的物体占⽤更多的像素，所以需要更⼤的卷积窗口来捕获物体。第⼆层中的卷积窗口形状减小到$5 \times 5$，之后全采⽤$3 \times 3$。此外，第⼀、第⼆和第五个卷积层之后都使⽤了窗口形状为$3 \times 3$、步幅为2的最⼤池化层。而且，`AlexNet`使⽤的卷积通道数也⼤于`LeNet`中的卷积通道数数⼗倍。

    紧接着最后⼀个卷积层的是两个输出个数为4096的全连接层。这两个巨⼤的全连接层带来将近1 GB的模型参数。由于早期显存的限制，最早的AlexNet使⽤双数据流的设计使⼀个GPU只需要处理⼀半模型。幸运的是，显存在过去⼏年得到了⻓⾜的发展，因此通常我们不再需要这样的特别设计了

2. `AlexNet`将`sigmoid`激活函数改成了更加简单的`ReLU`激活函数。⼀⽅⾯，`ReLU`激活函数的计算更简单，例如它并没有`sigmoid`激活函数中的求幂运算。另⼀⽅⾯，`ReLU`激活函数在不同的参数初始化⽅法下使模型更容易训练。这是由于当`sigmoid`激活函数输出极接近0或1时，这些区域的梯度⼏乎为0，从而造成反向传播⽆法继续更新部分模型参数；而`ReLU`激活函数在正区间的梯度恒为1。因此，若模型参数初始化不当，`sigmoid`函数可能在正区间得到⼏乎为0的梯度，从而令模型⽆法得到有效训练。

3. `AlexNet`通过丢弃法来控制全连接层的模型复杂度。而`LeNet`并没有使⽤丢弃法

4. `AlexNet`引⼊了⼤量的图像增⼴，如翻转、裁剪和颜⾊变化，从而进⼀步扩⼤数据集来缓解过拟合

下⾯我们实现稍微简化过的`AlexNet`。

In [9]:
net = nn.Sequential() 

# 使⽤较⼤的11 x 11窗⼝来捕获物体。同时使⽤步幅4来较⼤幅度减⼩输出⾼和宽。这⾥使⽤的输出通 
# 道数⽐LeNet中的也要⼤很多
net.add(
    nn.Conv2D(96, kernel_size=11, strides=4, activation='relu'),
    nn.MaxPool2D(pool_size=3, strides=2),
    # 减⼩卷积窗⼝，使⽤填充为2来使得输⼊与输出的⾼和宽⼀致，且增⼤输出通道数
    nn.Conv2D(256, kernel_size=5, padding=2, activation='relu'),
    nn.MaxPool2D(pool_size=3, strides=2),
    # 连续3个卷积层，且使⽤更⼩的卷积窗⼝。除了最后的卷积层外，进⼀步增⼤了输出通道数。 
    # 前两个卷积层后不使⽤池化层来减⼩输⼊的⾼和宽
    nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
    nn.Conv2D(384, kernel_size=3, padding=1, activation='relu'),
    nn.Conv2D(256, kernel_size=3, padding=1, activation='relu'),
    nn.MaxPool2D(pool_size=3, strides=2),
    # 这⾥全连接层的输出个数⽐LeNet中的⼤数倍。使⽤丢弃层来缓解过拟合
    nn.Dense(4096, activation="relu"), 
    nn.Dropout(0.5),
    nn.Dense(4096, activation="relu"), 
    nn.Dropout(0.5),
    # 输出层。由于这⾥使⽤Fashion-MNIST，所以⽤类别数为10，⽽⾮论⽂中的1000 
    nn.Dense(10))

In [10]:
# 我们构造⼀个⾼和宽均为224的单通道数据样本来观察每⼀层的输出形状
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize() 
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv2 output shape:	 (1, 96, 54, 54)
pool2 output shape:	 (1, 96, 26, 26)
conv3 output shape:	 (1, 256, 26, 26)
pool3 output shape:	 (1, 256, 12, 12)
conv4 output shape:	 (1, 384, 12, 12)
conv5 output shape:	 (1, 384, 12, 12)
conv6 output shape:	 (1, 256, 12, 12)
pool4 output shape:	 (1, 256, 5, 5)
dense3 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense4 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense5 output shape:	 (1, 10)


### 5.6.3 读取数据
虽然论⽂中`AlexNet`使⽤ImageNet数据集，但因为ImageNet数据集训练时间较⻓，我们仍⽤Fashion-MNIST数据集来演⽰`AlexNet`。 

读取数据的时候我们额外做了⼀步将图像⾼和宽扩⼤到AlexNet使⽤的图像⾼和宽224。这个可以通过Resize实例来实现。也就是说，我们在`ToTensor`实例前使⽤`Resize`实例，然后使⽤`Compose`实例来将这两个变换串联以⽅便调⽤。

In [11]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join( '~', '.mxnet', 'datasets', 'fashion-mnist')):
    root = os.path.expanduser(root) # 展开⽤⼾路径'~' 
    transformer = [] 
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)] 
    transformer += [gdata.vision.transforms.ToTensor()] 
    transformer = gdata.vision.transforms.Compose(transformer) 
    mnist_train = gdata.vision.FashionMNIST(root=root, train=True) 
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False) 
    num_workers = 4 
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True,num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle=False, num_workers=num_workers) 
    return train_iter, test_iter

batch_size = 128 
# 如出现“out of memory”的报错信息，可减⼩batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

### 5.6.4 训练
这时候我们可以开始训练`AlexNet`了。相对于上⼀节的`LeNet`，这⾥的主要改动是使⽤了更小的学习率。

[todo]
```python
# lr, num_epochs, ctx = 0.01, 5, d2l.try_gpu()
lr, num_epochs, ctx = 0.01, 5, mx.cpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)
```

`AlexNet`跟`LeNet`结构类似，但使⽤了更多的卷积层和更⼤的参数空间。它是浅层神经⽹络和深度神经⽹络的分界线。虽然看上去`AlexNet`的实现⽐`LeNet`的实现也就多了⼏⾏代码而已，但这个观念上的转变和真正优秀实验结果的产⽣令学术界付出了很多年。


## 5.7 使⽤重复元素的⽹络（VGG）
`AlexNet`在`LeNet`的基础上增加了3个卷积层。但`AlexNet`作者对它们的卷积窗口、输出通道数和构造顺序均做了⼤量的调整。虽然`AlexNet`指明了深度卷积神经⽹络可以取得出⾊的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的⽹络。我们将在本章的后续⼏节⾥介绍⼏种不同的深度⽹络设计思路。

`VGG`的名字来源于论⽂作者所在的实验室`Visual Geometry Group`。`VGG`提出了可以通过重复使⽤简单的基础块来构建深度模型的思路。

### 5.7.1 VGG块
`VGG`块的组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用`vgg_block`函数来实现这个基础的`VGG`块，它可以指定卷积层的数量`num_convs`和输出通道数`num_channels`。

In [12]:
def vgg_block(num_convs, num_channels):
    blk = nn.Sequential() 
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels, kernel_size=3, padding=1, activation='relu')) 
    blk.add(nn.MaxPool2D(pool_size=2, strides=2)) 
    return blk

### 5.7.2 VGG⽹络
`VGG`网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个`vgg_block`，其超参数由变量`conv_arch`定义。该变量指定了每个`VGG`块里卷积层个数和输出通道数。全连接模块则跟`AlexNet`中的一样。

我们构造一个`VGG`网络。它有5个卷积块，前2块使用单卷积层，而后3块使用双卷积层。第一块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个网络使用了8个卷积层和3个全连接层，所以经常被称为`VGG-11`。

In [13]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    net = nn.Sequential() 
    # 卷积层部分 
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels)) 
    # 全连接层部分
    net.add(
        nn.Dense(4096, activation='relu'), 
        nn.Dropout(0.5),
        nn.Dense(4096, activation='relu'), 
        nn.Dropout(0.5),
        nn.Dense(10)) 
    return net

net = vgg(conv_arch)

下⾯构造⼀个⾼和宽均为224的单通道数据样本来观察每⼀层的输出形状。

In [14]:
net.initialize()
X = nd.random.uniform(shape=(1, 1, 224, 224))
for blk in net:
    X = blk(X) 
    print(blk.name, 'output shape:\t', X.shape)

sequential3 output shape:	 (1, 64, 112, 112)
sequential4 output shape:	 (1, 128, 56, 56)
sequential5 output shape:	 (1, 256, 28, 28)
sequential6 output shape:	 (1, 512, 14, 14)
sequential7 output shape:	 (1, 512, 7, 7)
dense6 output shape:	 (1, 4096)
dropout2 output shape:	 (1, 4096)
dense7 output shape:	 (1, 4096)
dropout3 output shape:	 (1, 4096)
dense8 output shape:	 (1, 10)


可以看到，每次我们将输⼊的⾼和宽减半，直到最终⾼和宽变成7后传⼊全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口⼤小⼀样，所以每层的模型参数尺⼨和计算复杂度与输⼊⾼、输⼊宽、输⼊通道数和输出通道数的乘积成正⽐。VGG这种⾼和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺⼨和计算复杂度。

### 5.7.3 获取数据和训练模型
因为`VGG-11`计算上⽐`AlexNet`更加复杂，出于测试的⽬的我们构造⼀个通道数更小，或者说更窄的⽹络在`Fashion-MNIST`数据集上进⾏训练。

In [15]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch] 
net = vgg(small_conv_arch)

除了使⽤了稍⼤些的学习率，模型训练过程与上⼀节的`AlexNet`中的类似。

[TODO]
```python
lr, num_epochs, batch_size, ctx = 0.05, 5, 128, mx.cpu()
net.initialize(ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)
```

## 5.8 ⽹络中的⽹络(NiN)
`LeNet`、`AlexNet`和`VGG`在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，`AlexNet`和`VGG`对`LeNet`的改进主要在于如何对这两个模块加宽(增加通道数)和加深。

`⽹络中的⽹络`(NiN)提出了另外⼀个思路，即串联多个由卷积层和`全连接`层构成的小⽹络来构建⼀个深层⽹络。

### 5.8.1 NiN块
卷积层的输⼊和输出通常是四维数组(样本、通道、⾼、宽)，而全连接层的输⼊和输出则通常是⼆维数组(样本、特征)。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。

我们在`多输⼊通道和多输出通道`⼀节⾥介绍的$1 \times 1$卷积层。它可以看成全连接层，其中空间维度(⾼和宽)上的每个元素相当于样本，通道相当于特征。因此，`NiN`使⽤$1 \times 1$卷积层来替代全连接层，从而使空间信息能够⾃然传递到后⾯的层中去。`图5.7`对⽐了`NiN`同`AlexNet`和`VGG`等⽹络在结构上的主要区别。

<img src="images/05_07.png" style="width:500px;"/>

`NiN`块由⼀个卷积层加两个充当全连接层的$1 \times 1$卷积层串联而成。其中第⼀个卷积层的超参数可以⾃⾏设置，而第⼆和第三个卷积层的超参数⼀般是固定的。

In [16]:
def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential() 
    blk.add(
        nn.Conv2D(num_channels, kernel_size, strides, padding, activation='relu'),
        nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
        nn.Conv2D(num_channels, kernel_size=1, activation='relu')) 
    return blk

### 5.8.2 NiN模型
`NiN`是在`AlexNet`问世不久后提出的。它们的卷积层设定有类似之处。`NiN`使用卷积窗口形状分别为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，相应的输出通道数也与`AlexNet`中的一致。每个`NiN`块后接一个步幅为2、窗口形状为$3\times 3$的最大池化层。

除使用`NiN`块以外，`NiN`还有一个设计与`AlexNet`显著不同：`NiN`去掉了`AlexNet`最后的3个全连接层，使用了输出通道数等于标签类别数的`NiN`块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。`NiN`的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。然而，该设计有时会造成获得有效模型的训练时间的增加。

In [17]:
net = nn.Sequential()
net.add(
    nin_block(96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2D(pool_size=3, strides=2),
    nin_block(256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2D(pool_size=3, strides=2),
    nin_block(384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2D(pool_size=3, strides=2), 
    nn.Dropout(0.5),
    # 标签类别数是10 
    nin_block(10, kernel_size=3, strides=1, padding=1),
    # 全局平均池化层将窗⼝形状⾃动设置成输⼊的⾼和宽 
    nn.GlobalAvgPool2D(), 
    # 将四维的输出转成⼆维的输出，其形状为(批量⼤⼩, 10) 
    nn.Flatten())

我们构建⼀个数据样本来查看每⼀层的输出形状。

In [18]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize() 
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential15 output shape:	 (1, 96, 54, 54)
pool15 output shape:	 (1, 96, 26, 26)
sequential16 output shape:	 (1, 256, 26, 26)
pool16 output shape:	 (1, 256, 12, 12)
sequential17 output shape:	 (1, 384, 12, 12)
pool17 output shape:	 (1, 384, 5, 5)
dropout6 output shape:	 (1, 384, 5, 5)
sequential18 output shape:	 (1, 10, 5, 5)
pool18 output shape:	 (1, 10, 1, 1)
flatten0 output shape:	 (1, 10)


### 5.8.3 获取数据和训练模型
我们依然使⽤Fashion-MNIST数据集来训练模型。`NiN`的训练与`AlexNet`和`VGG`的类似，但这⾥使⽤的学习率更⼤。

[todo]
```python
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, mx.cpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)
```

## 5.9 含并⾏连结的⽹络(GoogLeNet)
`GoogLeNet`在2014年的ImageNet图像识别挑战赛中⼤放异彩。它虽然在名字上向`LeNet`致敬，但在⽹络结构上已经很难看到`LeNet`的影⼦。`GoogLeNet`吸收了`NiN`中⽹络串联⽹络的思想，并在此基础上做了很⼤改进。在随后的⼏年⾥，研究⼈员对`GoogLeNet`进⾏了数次改进，本节将介绍这个模型系列的第⼀个版本。

### 5.9.1 Inception 块
`GoogLeNet`中的基础卷积块叫作`Inception`块，得名于同名电影《盗梦空间》。与`NiN`块相⽐，这个基础块在结构上更加复杂，如`图5.8`所⽰。

<img src="images/05_08.png" style="width:500px;"/>

由`图5.8`可以看出，`Inception`块里有4条并行的线路。前3条线路使用窗口大小分别是$1\times 1$、$3\times 3$和$5\times 5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times 1$卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用$3\times 3$最大池化层，后接$1\times 1$卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

`Inception`块中可以自定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [19]:
class Inception(nn.Block):
    # c1 - c4为每条线路⾥的层的输出通道数
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs) 
        # 线路1，单1 x 1卷积层 
        self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu') 
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
        self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu') 
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
        self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu') 
        # 线路4，3 x 3最⼤池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
        self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')

    def forward(self, x):
        p1 = self.p1_1(x) 
        p2 = self.p2_2(self.p2_1(x)) 
        p3 = self.p3_2(self.p3_1(x)) 
        p4 = self.p4_2(self.p4_1(x)) 
        return nd.concat(p1, p2, p3, p4, dim=1) # 在通道维上连结输出

### 5.9.2 GoogLeNet模型
`GoogLeNet`跟`VGG`一样，在主体卷积部分中使用5个模块(block)，每个模块之间使用步幅为2的$3\times 3$最大池化层来减小输出高宽。第一模块使用一个64通道的$7\times 7$卷积层。

In [20]:
b1 = nn.Sequential()
b1.add(
    nn.Conv2D(64, kernel_size=7, strides=2, padding=3, activation='relu'),
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第二模块使用2个卷积层：首先是64通道的$1\times 1$卷积层，然后是将通道增大3倍的$3\times 3$卷积层。它对应Inception块中的第二条线路。

In [21]:
b2 = nn.Sequential()
b2.add(
    nn.Conv2D(64, kernel_size=1, activation='relu'),
    nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第三模块串联2个完整的`Inception`块。第一个`Inception`块的输出通道数为$64+128+32+32=256$，其中4条线路的输出通道数比例为$64:128:32:32=2:4:1:1$。其中第二、第三条线路先分别将输入通道数减小至$96/192=1/2$和$16/192=1/12$后，再接上第二层卷积层。第二个Inception块输出通道数增至$128+192+96+64=480$，每条线路的输出通道数之比为$128:192:96:64 = 4:6:3:2$。其中第二、第三条线路先分别将输入通道数减小至$128/256=1/2$和$32/256=1/8$。

In [22]:
b3 = nn.Sequential()
b3.add(
    Inception(64, (96, 128), (16, 32), 32), 
    Inception(128, (128, 192), (32, 96), 64), 
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第四模块更加复杂。它串联了5个`Inception`块，其输出通道数分别是：
+ $192+208+48+64=512$
+ $160+224+64+64=512$
+ $128+256+64+64=512$
+ $112+288+64+64=528$
+ $256+320+128+128=832$

这些线路的通道数分配和第三模块中的类似，首先含$3\times 3$卷积层的第二条线路输出最多通道，其次是仅含$1\times 1$卷积层的第一条线路，之后是含$5\times 5$卷积层的第三条线路和含$3\times 3$最大池化层的第四条线路。其中第二、第三条线路都会先按比例减小通道数。这些比例在各个`Inception`块中都略有不同。

In [23]:
b4 = nn.Sequential()
b4.add(
    Inception(192, (96, 208), (16, 48), 64), 
    Inception(160, (112, 224), (24, 64), 64), 
    Inception(128, (128, 256), (24, 64), 64), 
    Inception(112, (144, 288), (32, 64), 64),
    Inception(256, (160, 320), (32, 128), 128), 
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第五模块有输出通道数为$256+320+128+128=832$和$384+384+128+128=1024$的两个`Inception`块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，该模块同`NiN`一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。

In [24]:
b5 = nn.Sequential() 
b5.add(
    Inception(256, (160, 320), (32, 128), 128),
    Inception(384, (192, 384), (48, 128), 128), 
    nn.GlobalAvgPool2D())

net = nn.Sequential() 
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

`GoogLeNet`模型的计算复杂，而且不如`VGG`那样便于修改通道数。本节⾥我们将输⼊的⾼和宽从224降到96来简化计算。下⾯演⽰各个模块之间的输出的形状变化。

In [25]:
X = nd.random.uniform(shape=(1, 1, 96, 96))
net.initialize() 
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential19 output shape:	 (1, 64, 24, 24)
sequential20 output shape:	 (1, 192, 12, 12)
sequential21 output shape:	 (1, 480, 6, 6)
sequential22 output shape:	 (1, 832, 3, 3)
sequential23 output shape:	 (1, 1024, 1, 1)
dense12 output shape:	 (1, 10)


### 5.9.3 获取数据和训练模型
我们使⽤⾼和宽均为96像素的图像来训练`GoogLeNet`模型。 训练使⽤的图像依然来⾃`FashionMNIST`数据集。

[todo]
```python
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)
```

##### 小结
+ `Inception`块相当于一个有4条线路的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用$1\times 1$卷积层减少通道数从而降低模型复杂度
+ `GoogLeNet`将多个设计精细的`Inception`块和其他层串联起来。其中`Inception`块的通道数分配之比是在`ImageNet`数据集上通过大量的实验得来的
+ `GoogLeNet`和它的后继者们一度是ImageNet上最高效的模型之一：在类似的测试精度下，它们的计算复杂度往往更低



## 5.10 批量归⼀化
本节我们介绍`批量归⼀化`(batch normalization)层，它能让较深的神经⽹络的训练变得更加容易。

在`实战Kaggle⽐赛：预测房价`⼀节⾥，我们对输⼊数据做了标准化处理：处理后的任意⼀个特征在数据集中所有样本上的均值为0、标准差为1。标准化处理输⼊数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就⾜够有效了。随着模型训练的进⾏，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经⽹络来说，即使输⼊数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归⼀化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归⼀化利⽤小批量上的均值和标准差，不断调整神经⽹络中间输出，从而使整个神经⽹络在各层的中间输出的数值更稳定。批量归⼀化和残差⽹络为训练和设计深度模型提供了两类重要思路。

### 5.10.1 批量归⼀化层
对全连接层和卷积层做批量归⼀化的⽅法稍有不同。下⾯我们将分别介绍这两种情况下的批量归⼀化。

##### 对全连接层做批量归一化
通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为$\boldsymbol{u}$，权重参数和偏差参数分别为$\boldsymbol{W}$和$\boldsymbol{b}$，激活函数为$\phi$。设批量归一化的运算符为$\text{BN}$。那么，使用批量归一化的全连接层的输出为
+ $\phi(\text{BN}(\boldsymbol{x}))$

其中批量归一化输入$\boldsymbol{x}$由仿射变换
+ $\boldsymbol{x} = \boldsymbol{W\boldsymbol{u} + \boldsymbol{b}}$

得到。

考虑一个由$m$个样本组成的小批量，仿射变换的输出为一个新的小批量$\mathcal{B} = \{ \boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(m)} \}$。它们正是批量归一化层的输入。对于小批量$\mathcal{B}$中任意样本$\boldsymbol{x}^{(i)} \in \mathbb{R}^d, 1 \leq i \leq m$，批量归一化层的输出同样是$d$维向量

$$\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)}),$$

并由以下几步求得。首先，对小批量$\mathcal{B}$求均值和方差：
+ $\displaystyle\boldsymbol{\mu}_{\mathcal{B}} \leftarrow \frac{1}{m}\sum_{i = 1}^{m} \boldsymbol{x}^{(i)}$ 

+ $\displaystyle\boldsymbol{\sigma}_{\mathcal{B}}^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B})^2$

其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对$\boldsymbol{x}^{(i)}$标准化：
+ $\displaystyle{\hat{\boldsymbol{x}}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B}}{\sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}}$

这里$\epsilon > 0$是一个很小的常数，保证分母大于0。

在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，`拉伸`(scale)参数$\boldsymbol{\gamma}$和`偏移`(shift)参数 $\boldsymbol{\beta}$。这两个参数和$\boldsymbol{x}^{(i)}$形状相同，皆为$d$维向量。它们与$\hat{\boldsymbol{x}}^{(i)}$分别做按元素乘法（符号$\odot$）和加法计算：
+ ${\boldsymbol{y}}^{(i)} \leftarrow \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}$

至此，我们得到了$\boldsymbol{x}^{(i)}$的批量归一化的输出$\boldsymbol{y}^{(i)}$。 值得注意的是，可学习的拉伸和偏移参数保留了不对$\boldsymbol{x}^{(i)}$做批量归一化的可能：此时只需学出$\boldsymbol{\gamma} = \sqrt{\boldsymbol{\sigma}\mathcal{B}^2 + \epsilon}$和$\boldsymbol{\beta} = \boldsymbol{\mu}\mathcal{B}$。**我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化**。

##### 对卷积层做批量归一化
对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为标量。

设小批量中有$m$个样本。在单个通道上，假设卷积计算输出的高和宽分别为$p$和$q$。我们需要对该通道中$m \times p \times q$个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中$m \times p \times q$个元素的均值和方差。

##### 预测时的批量归一化
使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

### 5.10.2 从零开始实现
下⾯我们通过NDArray来实现批量归⼀化层。

In [26]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过autograd来判断当前模式是训练模式还是预测模式 
    if not autograd.is_training():
        # 如果是在预测模式下，直接使⽤传⼊的移动平均所得的均值和⽅差 
        X_hat = (X - moving_mean) / nd.sqrt(moving_var + eps) 
    else:
        assert len(X.shape) in (2, 4) 
        if len(X.shape) == 2:
            # 使⽤全连接层的情况，计算特征维上的均值和⽅差 
            mean = X.mean(axis=0) 
            var = ((X - mean) ** 2).mean(axis=0) 
        else:
            # 使⽤⼆维卷积层的情况，计算通道维上(axis=1)的均值和⽅差
            # 我们需要保持 X的形状以便后⾯可以做⼴播运算 
            mean = X.mean(axis=(0, 2, 3), keepdims=True) 
            var = ((X - mean) ** 2).mean(axis=(0, 2, 3), keepdims=True) 
        # 训练模式下⽤当前的均值和⽅差做标准化 
        X_hat = (X - mean) / nd.sqrt(var + eps) 
        # 更新移动平均的均值和⽅差 
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean 
        moving_var = momentum * moving_var + (1.0 - momentum) * var 
    Y = gamma * X_hat + beta # 拉伸和偏移 
    return Y, moving_mean, moving_var

定义一个`BatchNorm`层，它保存参与求梯度和迭代的拉伸参数`gamma`和偏移参数`beta`，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。`BatchNorm`实例所需指定的`num_features`参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。该实例所需指定的`num_dims`参数对于全连接层和卷积层来说分别为2和4。

In [27]:
class BatchNorm(nn.Block):
    def __init__(self, num_features, num_dims, **kwargs):
        super(BatchNorm, self).__init__(**kwargs) 
        if num_dims == 2:
            shape = (1, num_features) 
        else:
            shape = (1, num_features, 1, 1) 
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0 
        self.gamma = self.params.get('gamma', shape=shape, init=init.One()) 
        self.beta = self.params.get('beta', shape=shape, init=init.Zero()) 
        # 不参与求梯度和迭代的变量，全在内存上初始化成0 
        self.moving_mean = nd.zeros(shape) 
        self.moving_var = nd.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上 
        if self.moving_mean.context != X.context:
            self.moving_mean = self.moving_mean.copyto(X.context)
            self.moving_var = self.moving_var.copyto(X.context) 
        # 保存更新过的moving_mean和moving_var 
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma.data(), self.beta.data(), self.moving_mean, self.moving_var, eps=1e-5, momentum=0.9) 
        return Y

##### 使用批量归一化层的LeNet
下面我们修改`卷积神经网络(LeNet)`这一节介绍的`LeNet`模型，从而应用批量归一化层。

In [28]:
net = nn.Sequential()
net.add(
    nn.Conv2D(6, kernel_size=5), 
    BatchNorm(6, num_dims=4), 
    nn.Activation('sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(16, kernel_size=5), 
    BatchNorm(16, num_dims=4), 
    nn.Activation('sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120),
    BatchNorm(120, num_dims=2), 
    nn.Activation('sigmoid'), 
    nn.Dense(84), 
    BatchNorm(84, num_dims=2), 
    nn.Activation('sigmoid'), 
    nn.Dense(10))

训练修改后的模型。

In [29]:
# lr, num_epochs, batch_size, ctx = 1.0, 5, 256, d2l.try_gpu()
lr, num_epochs, batch_size, ctx = 1.0, 5, 256, mx.cpu()
net.initialize(ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on cpu(0)
epoch 1, loss 0.6507, train acc 0.768, test acc 0.844, time 154.0 sec
epoch 2, loss 0.3962, train acc 0.856, test acc 0.810, time 153.7 sec
epoch 3, loss 0.3483, train acc 0.873, test acc 0.850, time 153.7 sec
epoch 4, loss 0.3233, train acc 0.883, test acc 0.841, time 153.4 sec
epoch 5, loss 0.3026, train acc 0.891, test acc 0.880, time 154.8 sec


最后我们查看第一个批量归一化层学习到的拉伸参数`gamma`和偏移参数`beta`。

In [30]:
net[1].gamma.data().reshape((-1,)), net[1].beta.data().reshape((-1,))

(
 [1.0021073  1.472658   1.7876569  2.1462998  0.42314506 2.0245266 ]
 <NDArray 6 @cpu(0)>, 
 [-0.8766047 -1.4971851  0.8872622 -0.2025671 -0.2876061  1.3609908]
 <NDArray 6 @cpu(0)>)

### 5.10.3 简洁实现
`Gluon`中`nn`模块定义的`BatchNorm`类使⽤起来更加简单。它不需要指定⾃⼰定义的`BatchNorm`类中所需的`num_features`和`num_dims`参数值。 在Gluon中，这些参数值都将通过延后初始化而⾃动获取。

In [31]:
net = nn.Sequential() 
net.add(
    nn.Conv2D(6, kernel_size=5),
    nn.BatchNorm(), 
    nn.Activation('sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(16, kernel_size=5),
    nn.BatchNorm(), 
    nn.Activation('sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120), 
    nn.BatchNorm(), 
    nn.Activation('sigmoid'), 
    nn.Dense(84), 
    nn.BatchNorm(), 
    nn.Activation('sigmoid'), 
    nn.Dense(10))

# 训练
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on cpu(0)
epoch 1, loss 0.6620, train acc 0.765, test acc 0.848, time 23.1 sec
epoch 2, loss 0.3974, train acc 0.857, test acc 0.850, time 22.9 sec
epoch 3, loss 0.3457, train acc 0.875, test acc 0.872, time 23.0 sec
epoch 4, loss 0.3203, train acc 0.885, test acc 0.886, time 23.0 sec
epoch 5, loss 0.3027, train acc 0.891, test acc 0.874, time 23.0 sec


## 5.11 残差⽹络(ResNet)
让我们先思考⼀个问题：对神经⽹络模型添加新的层，充分训练后的模型是否只可能更有效地降低训练误差？理论上，原模型解的空间只是新模型解的空间的⼦空间。也就是说，如果我们能将新添加的层训练成恒等映射$f(x) = x$，新模型和原模型将同样有效。由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。然而在实践中，添加过多的层后训练误差往往不降反升。即使利⽤批量归⼀化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。

针对这⼀问题，何恺明等⼈提出了`残差⽹络`(ResNet)。它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经⽹络的设计。

### 5.11.1 残差块
让我们聚焦于神经网络局部。如`图5.9`所示，设输入为$\boldsymbol{x}$。假设我们希望学出的理想映射为$f(\boldsymbol{x})$，从而作为`图5.9`上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射$f(\boldsymbol{x})$，而右图虚线框中的部分则需要拟合出有关恒等映射的残差映射$f(\boldsymbol{x})-\boldsymbol{x}$。残差映射在实际中往往更容易优化。

以本节开头提到的恒等映射作为我们希望学出的理想映射$f(\boldsymbol{x})$。我们只需将`图5.9`中右图虚线框内上方的加权运算(如仿射)的权重和偏差参数学成0，那么$f(\boldsymbol{x})$即为恒等映射。实际中，当理想映射$f(\boldsymbol{x})$极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。

`图5.9`右图也是`ResNet`的基础块，即`残差块`(residual block)。在残差块中，输入可通过跨层的数据线路更快地向前传播。

<img src="images/05_09.png" style="width:600px;"/>

`ResNet`沿用了`VGG`全$3\times 3$卷积层的设计。残差块里首先有2个有相同输出通道数的$3\times 3$卷积层。每个卷积层后接一个批量归一化层和`ReLU`激活函数。然后我们将输入跳过这两个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求两个卷积层的输出与输入形状一样，从而可以相加。如果想改变通道数，就需要引入一个额外的$1\times 1$卷积层来将输入变换成需要的形状后再做相加运算。

残差块的实现如下。它可以设定输出通道数、是否使用额外的$1\times 1$卷积层来修改通道数以及卷积层的步幅。

In [32]:
class Residual(nn.Block): # 本类已保存在d2lzh包中⽅便以后使⽤
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs) 
        self.conv1 = nn.Conv2D(num_channels, kernel_size=3, padding=1, strides=strides) 
        self.conv2 = nn.Conv2D(num_channels, kernel_size=3, padding=1) 
        if use_1x1conv:
            self.conv3 = nn.Conv2D(num_channels, kernel_size=1, strides=strides) 
        else:
            self.conv3 = None 
        self.bn1 = nn.BatchNorm() 
        self.bn2 = nn.BatchNorm()

    def forward(self, X):
        Y = nd.relu(self.bn1(self.conv1(X))) 
        Y = self.bn2(self.conv2(Y)) 
        if self.conv3:
            X = self.conv3(X) 
        return nd.relu(Y + X)

下⾯我们来查看输⼊和输出形状⼀致的情况。

In [33]:
blk = Residual(3) 
blk.initialize()
X = nd.random.uniform(shape=(4, 3, 6, 6))
blk(X).shape

(4, 3, 6, 6)

我们也可以在增加输出通道数的同时减半输出的⾼和宽。

In [34]:
blk = Residual(6, use_1x1conv=True, strides=2) 
blk.initialize()
blk(X).shape

(4, 6, 3, 3)

### 5.11.2 ResNet模型
`ResNet`的前两层跟之前介绍的`GoogLeNet`中的一样：在输出通道数为64、步幅为2的$7\times 7$卷积层后接步幅为2的$3\times 3$的最大池化层。不同之处在于ResNet每个卷积层后增加的批量归一化层。

In [35]:
net = nn.Sequential()
net.add(
    nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
    nn.BatchNorm(), 
    nn.Activation('relu'), 
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

`GoogLeNet`在后⾯接了4个由`Inception`块组成的模块。`ResNet`则使⽤4个由残差块组成的模块，每个模块使⽤若⼲个同样输出通道数的残差块。第⼀个模块的通道数同输⼊通道数⼀致。由于之前已经使⽤了步幅为2的最⼤池化层，所以⽆须减小⾼和宽。之后的每个模块在第⼀个残差块⾥将上⼀个模块的通道数翻倍，并将⾼和宽减半。

下⾯我们来实现这个模块。注意，这⾥对第⼀个模块做了特别处理。

In [36]:
def resnet_block(num_channels, num_residuals, first_block=False):
    blk = nn.Sequential() 
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.add(Residual(num_channels, use_1x1conv=True, strides=2)) 
        else:
            blk.add(Residual(num_channels)) 
    return blk

接着我们为`ResNet`加⼊所有残差块。这⾥每个模块使⽤两个残差块。

In [37]:
net.add(
    resnet_block(64, 2, first_block=True), 
    resnet_block(128, 2), 
    resnet_block(256, 2), 
    resnet_block(512, 2))

最后，与`GoogLeNet`⼀样，加⼊全局平均池化层后接上全连接层输出。

In [38]:
net.add(nn.GlobalAvgPool2D(), nn.Dense(10))

这里每个模块里有4个卷积层(不计算$1\times 1$卷积层)，加上最开始的卷积层和最后的全连接层，共计18层。这个模型通常也被称为`ResNet-18`。通过配置不同的通道数和模块里的残差块数可以得到不同的ResNet模型，例如更深的含152层的`ResNet-152`。虽然`ResNet`的主体架构跟`GoogLeNet`的类似，但`ResNet`结构更简单，修改也更方便。这些因素都导致了`ResNet`迅速被广泛使用。

在训练`ResNet`之前，我们来观察一下输入形状在ResNet不同模块之间的变化。

In [39]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv101 output shape:	 (1, 64, 112, 112)
batchnorm12 output shape:	 (1, 64, 112, 112)
relu0 output shape:	 (1, 64, 112, 112)
pool37 output shape:	 (1, 64, 56, 56)
sequential28 output shape:	 (1, 64, 56, 56)
sequential29 output shape:	 (1, 128, 28, 28)
sequential30 output shape:	 (1, 256, 14, 14)
sequential31 output shape:	 (1, 512, 7, 7)
pool38 output shape:	 (1, 512, 1, 1)
dense19 output shape:	 (1, 10)


### 5.11.3 获取数据和训练模型
下⾯我们在Fashion-MNIST数据集上训练ResNet。

[todo]
```python
# lr, num_epochs, batch_size, ctx = 0.05, 5, 256, d2l.try_gpu()
lr, num_epochs, batch_size, ctx = 0.05, 5, 256, mx.cpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)
```

## 5.12 稠密连接⽹络(DenseNet)
`ResNet`中的跨层连接设计引申出了数个后续⼯作。本节我们介绍其中的⼀个：`稠密连接⽹络`(DenseNet)。它与`ResNet`的主要区别如`图5.10`所⽰。

<img src="images/05_10.png" style="width:600px;"/>

`图5.10`中将部分前后相邻的运算抽象为模块$A$和模块$B$。与`ResNet`的主要区别在于，`DenseNet`里模块$B$的输出不是像`ResNet`那样和模块$A$的输出相加，而是在通道维上连结。这样模块$A$的输出可以直接传入模块$B$后面的层。在这个设计里，模块$A$直接跟模块$B$后面的所有层连接在了一起。这也是它被称为`稠密连接`的原因。

`DenseNet`的主要构建模块是`稠密块`(dense block)和`过渡层`(transition layer)。前者定义了输入和输出是如何连结的，后者则用来控制通道数，使之不过大。

### 5.12.1 稠密块
`DenseNet`使⽤了`ResNet`改良版的`批量归⼀化、激活和卷积`结构(参⻅上⼀节的练习)，我们在`conv_block`函数⾥实现这个结构。

In [40]:
def conv_block(num_channels):
    blk = nn.Sequential() 
    blk.add(
        nn.BatchNorm(), 
        nn.Activation('relu'), 
        nn.Conv2D(num_channels, kernel_size=3, padding=1)) 
    return blk

稠密块由多个`conv_block`组成，每块使⽤相同的输出通道数。但在前向计算时，我们将每块的输⼊和输出在通道维上连结。

In [41]:
class DenseBlock(nn.Block):
    def __init__(self, num_convs, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs) 
        self.net = nn.Sequential()
        for _ in range(num_convs):
            self.net.add(conv_block(num_channels))

    def forward(self, X):
        for blk in self.net:
            Y = blk(X) 
            X = nd.concat(X, Y, dim=1) 
        return X

在下面的例子中，我们定义一个有2个输出通道数为10的卷积块。使用通道数为3的输入时，我们会得到通道数为$3+2\times 10=23$的输出。卷积块的通道数控制了输出通道数相对于输入通道数的增长，因此也被称为`增长率`(growth rate)。

In [42]:
blk = DenseBlock(2, 10) 
blk.initialize()
X = nd.random.uniform(shape=(4, 3, 8, 8))
Y = blk(X) 
Y.shape

(4, 23, 8, 8)

### 5.12.2 过渡层
由于每个稠密块都会带来通道数的增加，使⽤过多则会带来过于复杂的模型。过渡层⽤来控制模型复杂度。它通过$1 \times 1$卷积层来减小通道数，并使⽤步幅为2的平均池化层减半⾼和宽，从而进⼀步降低模型复杂度。

In [43]:
def transition_block(num_channels):
    blk = nn.Sequential() 
    blk.add(
        nn.BatchNorm(), 
        nn.Activation('relu'),
        nn.Conv2D(num_channels, kernel_size=1),
        nn.AvgPool2D(pool_size=2, strides=2)) 
    return blk

对上⼀个例⼦中稠密块的输出使⽤通道数为10的过渡层。此时输出的通道数减为10，⾼和宽均减半。

In [44]:
blk = transition_block(10) 
blk.initialize() 
blk(Y).shape

(4, 10, 4, 4)

### 5.12.3 DenseNet模型
我们来构造`DenseNet`模型。`DenseNet`⾸先使⽤同`ResNet`⼀样的单卷积层和最⼤池化层。

In [45]:
net = nn.Sequential()
net.add(
    nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
    nn.BatchNorm(), 
    nn.Activation('relu'), 
    nn.MaxPool2D(pool_size=3, strides=2, padding=1))

类似于`ResNet`接下来使用的4个残差块，`DenseNet`使用的是4个稠密块。同`ResNet`一样，我们可以设置每个稠密块使用多少个卷积层。这里我们设成4，从而与上一节的`ResNet-18`保持一致。稠密块里的卷积层通道数(即增长率)设为32，所以每个稠密块将增加128个通道。

`ResNet`里通过步幅为2的残差块在每个模块之间减小高和宽。这里我们则使用过渡层来减半高和宽，并减半通道数。

In [46]:
num_channels, growth_rate = 64, 32 # num_channels为当前的通道数 
num_convs_in_dense_blocks = [4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    net.add(DenseBlock(num_convs, growth_rate)) # 上⼀个稠密块的输出通道数 
    num_channels += num_convs * growth_rate # 在稠密块之间加⼊通道数减半的过渡层 
    if i != len(num_convs_in_dense_blocks) - 1:
        num_channels //= 2 
        net.add(transition_block(num_channels))

同`ResNet`⼀样，最后接上全局池化层和全连接层来输出。

In [47]:
net.add(
    nn.BatchNorm(), 
    nn.Activation('relu'), 
    nn.GlobalAvgPool2D(), 
    nn.Dense(10))

### 5.12.4 获取数据并训练模型
由于这⾥使⽤了⽐较深的⽹络，本节⾥我们将输⼊⾼和宽从224降到96来简化计算。

[todo]
```python
# lr, num_epochs, batch_size, ctx = 0.1, 5, 256, d2l.try_gpu()
lr, num_epochs, batch_size, ctx = 0.1, 5, 256, mx.cpu()
net.initialize(ctx=ctx, init=init.Xavier()) 
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}) 
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96) 
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)
```